# Problema de Planejamento de Produção de Camisetas
  
Uma fábrica fará vendas nas quatro semanas de finais de um campeonato.  
- Demanda semanal: $d = [5000,\,10000,\,30000,\,60000]$.  
- Capacidade regular por semana: $C^{\max} = 25000$ camisetas.  
- Nas semanas 1 e 2 é possível contratar **hora extra** (até $H^{\max}=10000$ por semana) — custo por unidade em hora extra = R\$ 2,80.  
- Custo de produção por unidade (regulares): R\$ 2,00 nas semanas 1 e 2; R\$ 2,50 nas semanas 3 e 4 (devido à concorrência).  
- Custo de estocagem: $e = 0.20$ R\$ por unidade por semana.  
- Estoque inicial assumido zero. Excesso produzido pode ser estocado para atender demandas futuras.  
- Objetivo: **minimizar o custo total** (produção + horas extras + estoque) satisfazendo as demandas semanais.

---

## Modelagem Matemática

### Parâmetros
Sejam semanas $t = 1,2,3,4$.

- $d_t$: demanda na semana $t$.  
  $d_1=5000,\; d_2=10000,\; d_3=30000,\; d_4=60000$.

- $C^{\max}$: capacidade regular semanal.  
  $$ C^{\max} = 25000 $$

- $H^{\max}_t$: capacidade de hora extra na semana $t$:  
  $$ H^{\max}_1 = H^{\max}_2 = 10000, \quad H^{\max}_3 = H^{\max}_4 = 0 $$

- Custo unitário de produção regular por semana $c^{reg}_t$:  
  $$ c^{reg}_1 = c^{reg}_2 = 2.00, \quad c^{reg}_3 = c^{reg}_4 = 2.50 $$

- Custo unitário de hora extra (somente $t=1,2$): $c^{ot}_t = 2.80$.  
- Custo de estocagem por semana: $e = 0.20$.  
- Estoque inicial: $s_0 = 0$.

---

### Variáveis de decisão
Para cada semana $t$:

- $c_t \geq 0$: quantidade produzida usando **capacidade regular**.  
- $h_t \geq 0$: quantidade produzida em **hora extra**.  
- $s_t \geq 0$: estoque ao final da semana $t$.

---

### Função objetivo

Minimizar o custo total:
$$
\min Z = \sum_{t=1}^{4} \Big( c^{reg}_t \, c_t + c^{ot}_t \, h_t + e \, s_t \Big)
$$

---

### Restrições

**Balanço de estoque (para cada semana $t$):**
$$
s_{t-1} + c_t + h_t - d_t = s_t, \quad s_0 = 0
$$

**Capacidade regular:**
$$
0 \leq c_t \leq C^{\max}, \quad t=1,\dots,4
$$

**Limite de hora extra:**
$$
0 \leq h_t \leq H^{\max}_t, \quad t=1,\dots,4
$$

**Não-negatividade do estoque:**
$$
s_t \geq 0, \quad t=1,\dots,4
$$

---


In [ ]:
# !pip install pulp

import pulp

# Parâmetros
semanas = [1,2,3,4]
d = {1:5000, 2:10000, 3:30000, 4:60000}
Cmax = 25000
Hmax = {1:10000, 2:10000, 3:0, 4:0}
custo_reg = {1:2.00, 2:2.00, 3:2.50, 4:2.50}
custo_extra = {1:2.80, 2:2.80, 3:0.0, 4:0.0}
e = 0.20
s0 = 0.0

# Modelo
modelo = pulp.LpProblem("Planejamento_Producao_Camisetas", pulp.LpMinimize)

# Variáveis
c = {t: pulp.LpVariable(f"c_{t}", lowBound=0, upBound=Cmax, cat="Continuous") for t in semanas}
h = {t: pulp.LpVariable(f"h_{t}", lowBound=0, upBound=Hmax[t], cat="Continuous") for t in semanas}
s = {t: pulp.LpVariable(f"s_{t}", lowBound=0, cat="Continuous") for t in semanas}

# Função objetivo
modelo += pulp.lpSum([ custo_reg[t]*c[t] + custo_extra[t]*h[t] + e*s[t] for t in semanas ]), "Custo_total"

#Balanço de estoque
for t in semanas:
    if t == 1:
        modelo += s0 + c[1] + h[1] - d[1] == s[1], f"Balanco_semana_{t}"
    else:
        modelo += s[t-1] + c[t] + h[t] - d[t] == s[t], f"Balanco_semana_{t}"

# Solver
modelo.solve(pulp.PULP_CBC_CMD(msg=1))

# Resultados
print("Status:", pulp.LpStatus[modelo.status])
print("Custo total (R$):", pulp.value(modelo.objective))
print("\nDecisões por semana:")
for t in semanas:
    print(f"Semana {t}: c={c[t].value():.0f}, h={h[t].value():.0f}, estoque final s={s[t].value():.0f}")


Status: Optimal
Custo total (R$): 258000.0

Decisões por semana:
Semana 1: c=25000, h=0, estoque final s=20000
Semana 2: c=25000, h=5000, estoque final s=40000
Semana 3: c=25000, h=0, estoque final s=35000
Semana 4: c=25000, h=0, estoque final s=0
